In [1]:
import sys, os

PROJECT_ROOT = r"C:\Assigment\2025-2026\Machine Learning\machine_learning_project" #thay bang file path root cua minh
SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_PATH:", SRC_PATH)

PROJECT_ROOT: C:\Assigment\2025-2026\Machine Learning\machine_learning_project
SRC_PATH: C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import data.before_model as before_model      
import data.data_preprocessing as data_preprocessing

In [3]:
df_daily = pd.read_csv(r'C:\Assigment\2025-2026\Machine Learning\machine_learning_project\dataset\raw\Hanoi_Daily_Selected.csv')

train_dataset, test_dataset = before_model.train_test_split(df_daily)
before_model_pipeline = before_model.before_model_pipeline
before_model_pipeline.fit(train_dataset)
train_dataset = before_model_pipeline.transform(train_dataset)
test_dataset = before_model_pipeline.transform(test_dataset)

Dropped columns due to constant value: []
Dropped columns due to zero variance: []


C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\data\data_preprocessing.py:109: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_df[col] = pd.to_numeric(data_df[col], errors='ignore')
c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Assigment\2025-2026\Machine Learning\machine_learning_project\src\data\data_preprocessing.py:109: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_df[col] = pd.to_numeric(data_df[col], errors='ignore')
C:\Assigment\2025-2026\Machine Learnin

In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import json  # Vẫn cần cho file tóm tắt (summary)
import datetime  # Vẫn cần cho file tóm tắt (summary)
import pickle  # Dùng để lưu model .pkl


class NpEncoder(json.JSONEncoder):
    """
    Custom JSON encoder để xử lý các kiểu dữ liệu của NumPy
    (vd: np.float64, np.int64) mà thư viện json tiêu chuẩn không xử lý được.
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


class MultiHorizonWalkForwardOptuna_XGBoost_Pipeline:
    def __init__(
        self,
        df,
        date_col,
        target_col,
        feature_cols,
        n_splits=5,
        test_size=30,
        mode="expanding",
        horizons=(1, 2, 3, 4, 5),  # t+1 ... t+5
    ):
        """
        ... (docstring) ...
        """
        self.df = df.copy().sort_values(date_col).reset_index(drop=True)
        self.date_col = date_col
        self.target_col = target_col
        self.feature_cols = feature_cols
        self.n_splits = n_splits
        self.test_size = test_size
        self.mode = mode
        self.horizons = tuple(horizons)

        self.walkfolds = {h: [] for h in self.horizons}
        self.walkfold_dates = {h: [] for h in self.horizons}

        self.final_models = {h: None for h in self.horizons}

        self.best_params = None
        self.fold_history = []

        self.X_cols = [
            c for c in self.feature_cols
            if c not in [
                self.date_col,
                self.target_col,
                self.target_col + "_next",
                self.date_col + "_next",
            ]
        ]

    # ======================== 1. TARGET SHIFT FOR ALL HORIZONS (TRAIN) ========================

    def add_target_shifts(self):
        """
        ... (docstring) ...
        """
        for h in self.horizons:
            self.df[f"{self.target_col}_h{h}"] = self.df[self.target_col].shift(-h)
            self.df[f"{self.date_col}_h{h}"] = self.df[self.date_col].shift(-h)

        target_cols = [f"{self.target_col}_h{h}" for h in self.horizons]
        self.df = self.df.dropna(subset=target_cols).reset_index(drop=True)

    # ======================== 1b. TARGET SHIFT CHO TEST SET ========================

    def prepare_test_dataset(self, test_df_raw):
        """
        ... (docstring) ...
        """
        test_df = test_df_raw.copy().sort_values(self.date_col).reset_index(drop=True)

        for h in self.horizons:
            test_df[f"{self.target_col}_h{h}"] = test_df[self.target_col].shift(-h)
            test_df[f"{self.date_col}_h{h}"] = test_df[self.date_col].shift(-h)

        target_cols = [f"{self.target_col}_h{h}" for h in self.horizons]
        test_df = test_df.dropna(subset=target_cols).reset_index(drop=True)
        return test_df

    # ======================== 2. CREATE WALK-FORWARD FOLDS ========================

    def create_walkforward_folds(self):
        """
        ... (docstring) ...
        """
        df_len = len(self.df)
        step = (df_len - self.test_size) // self.n_splits

        self.walkfolds = {h: [] for h in self.horizons}
        self.walkfold_dates = {h: [] for h in self.horizons}

        for i in range(self.n_splits):
            train_end = step * (i + 1)
            test_start = train_end + 91  # 91-day gap
            test_end = test_start + self.test_size
            if test_end > df_len:
                break

            if self.mode == "expanding":
                train_start = 0
            else:  # rolling
                train_start = max(0, train_end - step * 2)

            train_df = self.df.iloc[train_start:train_end]
            test_df = self.df.iloc[test_start:test_end]

            X_train = train_df[self.X_cols].reset_index(drop=True)
            X_val = test_df[self.X_cols].reset_index(drop=True)

            X_train_dates = train_df[self.date_col].reset_index(drop=True)
            X_val_dates = test_df[self.date_col].reset_index(drop=True)

            for h in self.horizons:
                y_train_h = train_df[f"{self.target_col}_h{h}"].reset_index(drop=True)
                y_val_h = test_df[f"{self.target_col}_h{h}"].reset_index(drop=True)

                y_train_dates_h = train_df[f"{self.date_col}_h{h}"].reset_index(drop=True)
                y_val_dates_h = test_df[f"{self.date_col}_h{h}"].reset_index(drop=True)

                self.walkfolds[h].append(
                    ((X_train, y_train_h), (X_val, y_val_h))
                )

                self.walkfold_dates[h].append(
                    ((X_train_dates, y_train_dates_h),
                     (X_val_dates, y_val_dates_h))
                )

    # ======================== 3. OPTUNA OBJECTIVE (SHARED PARAMS) ========================

    def create_objective(self):
        """
        ... (docstring) ...
        """
        def objective(trial):
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 6),
                'gamma': trial.suggest_float('gamma', 0.0, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 3, 10),
            }

            horizon_results = {}
            overall_rmse_list = []
            global_step = 0

            for h in self.horizons:
                scores = []
                fold_results = []

                for i, ((X_train, y_train), (X_val, y_val)) in enumerate(self.walkfolds[h]):

                    model = xgb.XGBRegressor(
                        **params,
                        n_estimators=800,
                        random_state=42,
                        tree_method='hist',
                        early_stopping_rounds=100
                    )

                    model.fit(
                        X_train,
                        y_train,
                        eval_set=[(X_val, y_val)],
                        verbose=False
                    )

                    preds = model.predict(X_val)
                    
                    # ĐÃ SỬA (FIXED): Lấy căn bậc hai của MSE để ra RMSE
                    rmse = np.sqrt(mean_squared_error(y_val, preds))

                    ss_res = np.sum((y_val - preds) ** 2)
                    ss_tot = np.sum((y_val - np.mean(y_val)) ** 2)
                    r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

                    scores.append(rmse)
                    fold_results.append({'rmse': rmse, 'r2': r2})

                    print(
                        f"Trial {trial.number}, Horizon h={h}, "
                        f"Fold {i+1}: RMSE={rmse:.4f}, R²={r2:.4f}"
                    )

                    trial.report(rmse, step=global_step)
                    if trial.should_prune():
                        print(f"Trial {trial.number} pruned at global_step={global_step}")
                        raise optuna.TrialPruned()
                    global_step += 1

                mean_rmse_h = float(np.mean(scores))
                horizon_results[h] = {
                    'mean_rmse': mean_rmse_h,
                    'fold_scores': fold_results,
                }
                overall_rmse_list.append(mean_rmse_h)

            overall_mean_rmse = float(np.mean(overall_rmse_list))
            print(
                f"Trial {trial.number} finished, "
                f"overall mean RMSE across horizons={overall_mean_rmse:.4f}\n"
            )

            self.fold_history.append({
                'trial': trial.number,
                'params': params,
                'horizon_results': horizon_results,
                'overall_mean_rmse': overall_mean_rmse,
            })

            return overall_mean_rmse

        return objective

    # ======================== 4. RUN OPTUNA ========================

    def run_optuna(self, n_trials=50):
        """
        ... (docstring) ...
        """
        objective_fn = self.create_objective()

        study = optuna.create_study(
            direction='minimize',
            pruner=optuna.pruners.MedianPruner(
                n_warmup_steps=5
            )
        )
        study.optimize(objective_fn, n_trials=n_trials)
        self.best_params = study.best_params
        return study

    # ======================== 5. TRAIN A FINAL MODEL PER HORIZON ========================

    def train_final_models(self):
        """
        ... (docstring) ...
        """
        if self.best_params is None:
            raise ValueError("You must run run_optuna() before train_final_models().")

        final_params = self.best_params.copy()
        final_params.update({
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'n_estimators': 800
        })

        X_full = self.df[self.X_cols]

        for h in self.horizons:
            y_full_h = self.df[f"{self.target_col}_h{h}"]

            model_h = xgb.XGBRegressor(
                **final_params,
                random_state=42,
                tree_method='hist'
            )

            model_h.fit(X_full, y_full_h, verbose=False)
            self.final_models[h] = model_h
            print(f"Final model for horizon {h} trained.")

    # ======================== 6. PREDICTION HELPERS ========================

    def predict_horizon(self, X_today, h):
        """
        ... (docstring) ...
        """
        if h not in self.horizons:
            raise ValueError(f"Horizon {h} not in configured horizons {self.horizons}.")
        if self.final_models[h] is None:
            raise ValueError(f"Model for horizon {h} is not trained. Call train_final_models() first.")

        X_input = X_today[self.X_cols]
        return self.final_models[h].predict(X_input)

    def predict_all_horizons(self, X_today):
        """
        ... (docstring) ...
        """
        results = {}
        for h in self.horizons:
            results[h] = self.predict_horizon(X_today, h)
        return results

    # ======================== 6b. BUILD PREDICTIONS FRAME (Plot) ========================

    def get_predictions_frame(self, df_with_shift, h):
        """
        ... (docstring) ...
        """
        if h not in self.horizons:
            raise ValueError(f"Horizon {h} not in configured horizons {self.horizons}.")
        if self.final_models[h] is None:
            raise ValueError(f"Model for horizon {h} is not trained. Call train_final_models() first.")

        X = df_with_shift[self.X_cols]
        y_true = df_with_shift[f"{self.target_col}_h{h}"]
        y_pred = self.final_models[h].predict(X)

        feature_time = df_with_shift[self.date_col]
        if f"{self.date_col}_h{h}" in df_with_shift.columns:
            target_time = df_with_shift[f"{self.date_col}_h{h}"]
        else:
            target_time = feature_time

        df_pred = pd.DataFrame({
            "feature_time": feature_time.values,
            "target_time": target_time.values,
            "y_true": y_true.values,
            "y_pred": y_pred,
        })
        return df_pred

    def plot_predictions(self, df_with_shift, h, use_target_time=True, n_points=None):
        """
        ... (docstring) ...
        """
        df_pred = self.get_predictions_frame(df_with_shift, h)

        df_plot = df_pred.copy()
        if use_target_time and "target_time" in df_plot.columns:
            df_plot["x"] = df_plot["target_time"]
        else:
            df_plot["x"] = df_plot["feature_time"]

        if n_points is not None:
            df_plot = df_plot.iloc[:n_points]

        plt.figure(figsize=(10, 4))
        plt.plot(df_plot["x"], df_plot["y_true"], label="True")
        plt.plot(df_plot["x"], df_plot["y_pred"], label="Predicted")
        plt.xlabel("Date")
        plt.ylabel(self.target_col)
        plt.title(f"Horizon h={h} forecast")
        plt.legend()
        plt.tight_layout()
        plt.show()

    # ======================== 7. EVAL ON TRAIN SET (CHECK OVERFIT) ========================

    def evaluate_train_models(self):
        """
        ... (docstring) ...
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("All horizon models must be trained before evaluation.")

        metrics = {}
        X_train_full = self.df[self.X_cols]

        for h in self.horizons:
            y_train_h = self.df[f"{self.target_col}_h{h}"]
            preds = self.final_models[h].predict(X_train_full)

            # ĐÃ SỬA (FIXED): Lấy căn bậc hai của MSE
            mse = mean_squared_error(y_train_h, preds)
            rmse = np.sqrt(mse)
            
            mae = mean_absolute_error(y_train_h, preds)

            eps = 1e-8
            mape = np.mean(np.abs((y_train_h - preds) / (y_train_h + eps))) * 100

            ss_res = np.sum((y_train_h - preds) ** 2)
            ss_tot = np.sum((y_train_h - np.mean(y_train_h)) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

            print(
                f"[TRAIN] Horizon h={h} -> "
                f"RMSE: {rmse:.4f}, R²: {r2:.4f}"
            )

            metrics[h] = {
                'rmse': rmse,
                'mae': mae,
                'mape': mape,
                'mse': mse,
                'r2': r2,
            }

        return metrics

    # ======================== 8. EVALUATION ON A TEST SET ========================

    def evaluate_final_models(self, test_df):
        """
        ... (docstring) ...
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("All horizon models must be trained before evaluation.")

        metrics = {}
        X_test = test_df[self.X_cols]

        for h in self.horizons:
            y_test_h = test_df[f"{self.target_col}_h{h}"]
            preds = self.final_models[h].predict(X_test)

            # ĐÃ SỬA (FIXED): Lấy căn bậc hai của MSE
            mse = mean_squared_error(y_test_h, preds)
            rmse = np.sqrt(mse)

            mae = mean_absolute_error(y_test_h, preds)

            eps = 1e-8
            mape = np.mean(np.abs((y_test_h - preds) / (y_test_h + eps))) * 100

            ss_res = np.sum((y_test_h - preds) ** 2)
            ss_tot = np.sum((y_test_h - np.mean(y_test_h)) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

            print(f"Horizon h={h} -> RMSE: {rmse:.4f}, R²: {r2:.4f}")

            metrics[h] = {
                'rmse': rmse,
                'mae': mae,
                'mape': mape,
                'mse': mse,
                'r2': r2,
            }

        return metrics

    # ======================== 9. SAVE FINAL MODELS (PICKLE) ========================

    def save_all_final_models_pkl(self, base_filename_prefix):
        """
        Lưu tất cả các model cuối cùng (final_models) ra đĩa dùng PICKLE (.pkl).
        """
        if any(self.final_models[h] is None for h in self.horizons):
            raise ValueError("Phải train model trước khi lưu. Hãy gọi train_final_models().")
        
        saved_paths = {}
        for h in self.horizons:
            model = self.final_models[h]
            filename = f"{base_filename_prefix}_h{h}.pkl"
            
            try:
                with open(filename, 'wb') as f: 
                    pickle.dump(model, f)
                saved_paths[h] = filename
                print(f"Model (Pickle) cho horizon {h} đã được lưu vào: {filename}")
            except Exception as e:
                print(f"Lỗi khi lưu file pickle {filename}: {e}")
        
        return saved_paths

    # ======================== 10. SAVE RESULTS SUMMARY (JSON) ========================
    
    def save_results_to_json(
        self,
        filename,
        model_name,
        train_metrics,
        test_metrics,
        saved_model_paths=None
    ):
        """
        Lưu kết quả tóm tắt (best param, metrics, timestamp,...) vào 1 file JSON.
        """
        
        results_summary = {
            "model_name": model_name,
            "run_timestamp_utc": datetime.utcnow().isoformat(),
            "target_column": self.target_col,
            "feature_columns": self.X_cols,
            "horizons": self.horizons,
            "cross_validation_setup": {
                "n_splits": self.n_splits,
                "test_size_per_fold": self.test_size,
                "mode": self.mode,
            },
            "optuna_best_params": self.best_params,
            "evaluation_train_set": train_metrics,
            "evaluation_test_set": test_metrics,
            "saved_model_files": saved_model_paths if saved_model_paths else {},
            "optuna_fold_history": self.fold_history 
        }

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(results_summary, f, indent=4, cls=NpEncoder)
            print(f"Kết quả tóm tắt đã được lưu vào: {filename}")
        except Exception as e:
            print(f"Lỗi khi lưu file JSON: {e}")

c:\Users\ASUS\anaconda3\envs\wp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
XGBoost_pipeline = MultiHorizonWalkForwardOptuna_XGBoost_Pipeline(
    df=train_dataset,
    date_col="datetime",
    target_col="temp",
    feature_cols=[col for col in train_dataset.columns if col not in ["datetime", "temp", "temp_next", "datetime_next"]],
    n_splits=5,
    test_size=365,
    mode="expanding",
    horizons=(1, 2, 3, 4, 5)
)

In [6]:
XGBoost_pipeline.add_target_shifts()
XGBoost_pipeline.create_walkforward_folds()

In [7]:
XGBoost_pipeline.run_optuna(n_trials=100)

[I 2025-11-17 11:05:25,027] A new study created in memory with name: no-name-55585e06-b3fc-426a-9cfa-c4aea6b70c42


Trial 0, Horizon h=1, Fold 1: RMSE=1.7248, R²=0.8646
Trial 0, Horizon h=1, Fold 2: RMSE=1.8729, R²=0.8497
Trial 0, Horizon h=1, Fold 3: RMSE=1.5661, R²=0.9043
Trial 0, Horizon h=1, Fold 4: RMSE=1.7267, R²=0.9001
Trial 0, Horizon h=2, Fold 1: RMSE=2.3073, R²=0.7595
Trial 0, Horizon h=2, Fold 2: RMSE=2.5495, R²=0.7213
Trial 0, Horizon h=2, Fold 3: RMSE=2.2231, R²=0.8099
Trial 0, Horizon h=2, Fold 4: RMSE=2.2579, R²=0.8288
Trial 0, Horizon h=3, Fold 1: RMSE=2.6472, R²=0.6857
Trial 0, Horizon h=3, Fold 2: RMSE=2.8468, R²=0.6513
Trial 0, Horizon h=3, Fold 3: RMSE=2.6073, R²=0.7396
Trial 0, Horizon h=3, Fold 4: RMSE=2.5187, R²=0.7864
Trial 0, Horizon h=4, Fold 1: RMSE=2.6748, R²=0.6799
Trial 0, Horizon h=4, Fold 2: RMSE=2.9198, R²=0.6335
Trial 0, Horizon h=4, Fold 3: RMSE=2.7296, R²=0.7148
Trial 0, Horizon h=4, Fold 4: RMSE=2.6117, R²=0.7698
Trial 0, Horizon h=5, Fold 1: RMSE=2.8302, R²=0.6415
Trial 0, Horizon h=5, Fold 2: RMSE=2.9433, R²=0.6290
Trial 0, Horizon h=5, Fold 3: RMSE=2.7519, R²=

[I 2025-11-17 11:05:56,939] Trial 0 finished with value: 2.4489576468242573 and parameters: {'learning_rate': 0.010226299051069821, 'max_depth': 3, 'gamma': 0.2843024866018208, 'reg_lambda': 9.536274411848495, 'reg_alpha': 5.9726286145707475, 'subsample': 0.8004030657208817, 'colsample_bytree': 0.5825086200845415, 'min_child_weight': 4}. Best is trial 0 with value: 2.4489576468242573.


Trial 0, Horizon h=5, Fold 4: RMSE=2.6696, R²=0.7591
Trial 0 finished, overall mean RMSE across horizons=2.4490

Trial 1, Horizon h=1, Fold 1: RMSE=1.7748, R²=0.8566
Trial 1, Horizon h=1, Fold 2: RMSE=1.8556, R²=0.8524
Trial 1, Horizon h=1, Fold 3: RMSE=1.6066, R²=0.8993
Trial 1, Horizon h=1, Fold 4: RMSE=1.7563, R²=0.8967
Trial 1, Horizon h=2, Fold 1: RMSE=2.2585, R²=0.7695
Trial 1, Horizon h=2, Fold 2: RMSE=2.6520, R²=0.6984
Trial 1, Horizon h=2, Fold 3: RMSE=2.2638, R²=0.8028
Trial 1, Horizon h=2, Fold 4: RMSE=2.2838, R²=0.8249
Trial 1, Horizon h=3, Fold 1: RMSE=2.5408, R²=0.7105
Trial 1, Horizon h=3, Fold 2: RMSE=2.9205, R²=0.6330
Trial 1, Horizon h=3, Fold 3: RMSE=2.6289, R²=0.7352
Trial 1, Horizon h=3, Fold 4: RMSE=2.5347, R²=0.7837
Trial 1, Horizon h=4, Fold 1: RMSE=2.5658, R²=0.7055
Trial 1, Horizon h=4, Fold 2: RMSE=2.9652, R²=0.6220
Trial 1, Horizon h=4, Fold 3: RMSE=2.7463, R²=0.7113
Trial 1, Horizon h=4, Fold 4: RMSE=2.6200, R²=0.7683
Trial 1, Horizon h=5, Fold 1: RMSE=2.83

[I 2025-11-17 11:06:14,924] Trial 1 finished with value: 2.466431168662011 and parameters: {'learning_rate': 0.025322979155211894, 'max_depth': 4, 'gamma': 0.2839287660282458, 'reg_lambda': 3.7967800631548645, 'reg_alpha': 2.699484013965173, 'subsample': 0.9802954031281349, 'colsample_bytree': 0.6323975818847702, 'min_child_weight': 7}. Best is trial 0 with value: 2.4489576468242573.


...

Trial 97, Horizon h=5, Fold 4: RMSE=2.6581, R²=0.7611
Trial 97 finished, overall mean RMSE across horizons=2.4193

Trial 98, Horizon h=1, Fold 1: RMSE=1.6483, R²=0.8764
Trial 98, Horizon h=1, Fold 2: RMSE=1.8459, R²=0.8540
Trial 98, Horizon h=1, Fold 3: RMSE=1.5499, R²=0.9063
Trial 98, Horizon h=1, Fold 4: RMSE=1.7194, R²=0.9010
Trial 98, Horizon h=2, Fold 1: RMSE=2.1717, R²=0.7869
Trial 98, Horizon h=2, Fold 2: RMSE=2.5472, R²=0.7218
Trial 98, Horizon h=2, Fold 3: RMSE=2.2282, R²=0.8090
Trial 98, Horizon h=2, Fold 4: RMSE=2.2492, R²=0.8302
Trial 98, Horizon h=3, Fold 1: RMSE=2.4821, R²=0.7237
Trial 98, Horizon h=3, Fold 2: RMSE=2.8292, R²=0.6556
Trial 98, Horizon h=3, Fold 3: RMSE=2.6171, R²=0.7376
Trial 98, Horizon h=3, Fold 4: RMSE=2.5422, R²=0.7824
Trial 98, Horizon h=4, Fold 1: RMSE=2.5377, R²=0.7119
Trial 98, Horizon h=4, Fold 2: RMSE=2.9507, R²=0.6257
Trial 98, Horizon h=4, Fold 3: RMSE=2.7136, R²=0.7182
Trial 98, Horizon h=4, Fold 4: RMSE=2.6180, R²=0.7687
Trial 98, Horizon h=5

[I 2025-11-17 11:35:11,396] Trial 98 finished with value: 2.4108078716271075 and parameters: {'learning_rate': 0.023054513745081964, 'max_depth': 3, 'gamma': 0.12270511175012579, 'reg_lambda': 2.4754370815322053, 'reg_alpha': 9.515075924539731, 'subsample': 0.5116564681340086, 'colsample_bytree': 0.5704451734974961, 'min_child_weight': 10}. Best is trial 84 with value: 2.403314691636111.


Trial 98, Horizon h=5, Fold 4: RMSE=2.6229, R²=0.7674
Trial 98 finished, overall mean RMSE across horizons=2.4108

Trial 99, Horizon h=1, Fold 1: RMSE=1.7040, R²=0.8679
Trial 99, Horizon h=1, Fold 2: RMSE=1.8459, R²=0.8540
Trial 99, Horizon h=1, Fold 3: RMSE=1.5636, R²=0.9046
Trial 99, Horizon h=1, Fold 4: RMSE=1.7124, R²=0.9018
Trial 99, Horizon h=2, Fold 1: RMSE=2.2110, R²=0.7791
Trial 99, Horizon h=2, Fold 2: RMSE=2.4790, R²=0.7365
Trial 99, Horizon h=2, Fold 3: RMSE=2.2343, R²=0.8079
Trial 99, Horizon h=2, Fold 4: RMSE=2.3012, R²=0.8222
Trial 99, Horizon h=3, Fold 1: RMSE=2.4490, R²=0.7310
Trial 99, Horizon h=3, Fold 2: RMSE=2.8179, R²=0.6583
Trial 99, Horizon h=3, Fold 3: RMSE=2.5629, R²=0.7483
Trial 99, Horizon h=3, Fold 4: RMSE=2.4685, R²=0.7948
Trial 99, Horizon h=4, Fold 1: RMSE=2.5830, R²=0.7015
Trial 99, Horizon h=4, Fold 2: RMSE=2.9367, R²=0.6292
Trial 99, Horizon h=4, Fold 3: RMSE=2.7151, R²=0.7178
Trial 99, Horizon h=4, Fold 4: RMSE=2.5987, R²=0.7721
Trial 99, Horizon h=5

[I 2025-11-17 11:35:21,776] Trial 99 finished with value: 2.4150303371265953 and parameters: {'learning_rate': 0.0429958766861205, 'max_depth': 3, 'gamma': 0.02924947897567777, 'reg_lambda': 1.0320851961101878, 'reg_alpha': 9.969060132769123, 'subsample': 0.5654282115388358, 'colsample_bytree': 0.6428962185916485, 'min_child_weight': 10}. Best is trial 84 with value: 2.403314691636111.


Trial 99, Horizon h=5, Fold 4: RMSE=2.6607, R²=0.7607
Trial 99 finished, overall mean RMSE across horizons=2.4150



In [8]:
XGBoost_pipeline.train_final_models()

Final model for horizon 1 trained.
Final model for horizon 2 trained.
Final model for horizon 3 trained.
Final model for horizon 4 trained.
Final model for horizon 5 trained.


In [9]:
# Train metrics
train_metrics = XGBoost_pipeline.evaluate_train_models()
train_metrics

[TRAIN] Horizon h=1 -> RMSE: 1.1708, R²: 0.9475
[TRAIN] Horizon h=2 -> RMSE: 1.5399, R²: 0.9093
[TRAIN] Horizon h=3 -> RMSE: 1.6437, R²: 0.8966
[TRAIN] Horizon h=4 -> RMSE: 1.6308, R²: 0.8982
[TRAIN] Horizon h=5 -> RMSE: 1.6423, R²: 0.8968


{1: {'rmse': np.float64(1.1707950685890531),
  'mae': 0.9097385533856804,
  'mape': np.float64(3.9047609315578566),
  'mse': 1.3707610926324456,
  'r2': np.float64(0.9475410032611999)},
 2: {'rmse': np.float64(1.5399174485082945),
  'mae': 1.1921388201350331,
  'mape': np.float64(5.1892627154767785),
  'mse': 2.3713457482202958,
  'r2': np.float64(0.9092606966991553)},
 3: {'rmse': np.float64(1.6437230929844828),
  'mae': 1.2985613474544482,
  'mape': np.float64(5.687264909859229),
  'mse': 2.701825606410475,
  'r2': np.float64(0.8966196501988548)},
 4: {'rmse': np.float64(1.6308412570631055),
  'mae': 1.3052086959047486,
  'mape': np.float64(5.702796526051049),
  'mse': 2.6596432057391697,
  'r2': np.float64(0.8982234744844875)},
 5: {'rmse': np.float64(1.642295801329434),
  'mae': 1.3080165989781507,
  'mape': np.float64(5.715439644070334),
  'mse': 2.697135499064288,
  'r2': np.float64(0.8967788368237757)}}

In [10]:
# Test metrics
test_df_shifted = XGBoost_pipeline.prepare_test_dataset(test_dataset)
test_metrics = XGBoost_pipeline.evaluate_final_models(test_df_shifted)

Horizon h=1 -> RMSE: 1.6457, R²: 0.8968
Horizon h=2 -> RMSE: 2.1429, R²: 0.8251
Horizon h=3 -> RMSE: 2.3181, R²: 0.7954
Horizon h=4 -> RMSE: 2.3886, R²: 0.7825
Horizon h=5 -> RMSE: 2.4149, R²: 0.7763


In [11]:
test_metrics

{1: {'rmse': np.float64(1.6457485674102432),
  'mae': 1.3000574212611657,
  'mape': np.float64(5.718517024274549),
  'mse': 2.7084883471328682,
  'r2': np.float64(0.8967907274198992)},
 2: {'rmse': np.float64(2.1428995906733355),
  'mae': 1.689180714801783,
  'mape': np.float64(7.56992495875285),
  'mse': 4.592018655707949,
  'r2': np.float64(0.8250957362472163)},
 3: {'rmse': np.float64(2.318113795974372),
  'mae': 1.8308525629628571,
  'mape': np.float64(8.240096479058158),
  'mse': 5.373651571086714,
  'r2': np.float64(0.795397576393617)},
 4: {'rmse': np.float64(2.388640124901085),
  'mae': 1.8815491130790762,
  'mape': np.float64(8.507366028914868),
  'mse': 5.705601646287472,
  'r2': np.float64(0.7824921692366025)},
 5: {'rmse': np.float64(2.414940909952376),
  'mae': 1.8892677968306821,
  'mape': np.float64(8.55985574213154),
  'mse': 5.83193959856161,
  'r2': np.float64(0.7762632957169942)}}